# **Phases 1 and 2 Revisited**

## *Prompt*

**Microsoft's Making Moves into Movies**

>* What types of movies are performing the best a the box offices?
>* What actions should they take based on the data?

## *Questions*

---
**What questions will guide my process?**

>* **Q1:What genres show the best performance?**
    * What is the best way to evaluate the *profitability* of the movies?
        * Gross
        * Profit
        * ROI
    * What is the best way to determine the best *quality* of each movie?
>
>
>* **Q2:How do we determine the "best" features?**
    * Which features would have the greatest impact on each of the performance metrics?
    * How to optimize for all three metrics?
>
>
>* **Q3:What release times are best for gross value?**
    * Seasons vs. Quarters?
        * *Would it make more sense either way? Could there be a benefit for inferences/predictions via LinReg modeling?*
>
>
>* **Q4:Which directors show the strongest/weakest performance?**
    * Get director names, ID
    * Add names to dataframe including the movie ID and gross performance/highest reviews
>
>
>* **Q5:What features would give the strongest indications of performance?**
>
>
>* **Q6:How can we determine whether or not a movie is "successful" or not?**

---

**Goals:**
>
>* **Determine KPIs**
    * How to measure a movie's success?
>
>
>* **Use linear regression modeling to determine the top 5 features for each target.**
    * To what degree do the features overlap?
    * What would be the top 3-5 features on which to focus to create the "best" movie (balancing all of the metrics)?

---

## 🎇 **WIP Current Status** 🎇

>*Update this section after each coding session - quick review/AAG overview of what is done and what is do-do*
>
>*Imagine its a scrum/kanban board in the notebook*

### **Done**

>* **Data Cleaning:** pulled, df'ed, coverted datatypes, filled "N/A" values
>
>
>* **FE:** 
>   * Months, Seasons, Quarters
>   * Genres to list, then to individual rows
>   * Profit and ROI metrics
>
>
>* **Visualizations:**
>   * Seasonal performance - gross, profit, ROI

### ✨**To-Do**✨

>* MultiCollinearity and Correlations
>   * ✨ Which features to compare against each other?
>   * ✨ How to create the matrix with these features?
>
>
>* Statistical Testing
>
>
>* LinReg
>   * ✨ OHE for inferential or not?
>   * ✨
>
>
---

## *Process*

>1. Import .csv's
>2. Clean data
>3. EDA w. visuals
>4. **Determine initial insights and actions**
>5. Create new features
>6. Test for correlations/multicollinearity
>7. Perform statistical testing
>8. Create LinReg model for **inference**
>9. Create LinReg model for **predictions**
>10. **Present final results for inferences, predictions**
    1. Include initial, final insights and recommendations

# **Imports**

## Packages

In [1]:
## Accessing stored data
import csv
import os,glob

## Data exploration and statistics
import pandas as pd
import numpy as np

## Creating Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# Modeling - Statsmodels
import statsmodels.formula.api as smf
import statsmodels.api as sms

# Modeling - SKLearn
from sklearn import metrics, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import RFECV

## Settings
%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set_context('notebook', font_scale=1.25)

## Data

In [2]:
## Creating list of files to loop through for data
data_folder = 'zippedData/'
data_files = glob.glob(f'{data_folder}*.csv*')
data_files

['zippedData\\bom.movie_gross.csv.gz',
 'zippedData\\imdb.name.basics.csv.gz',
 'zippedData\\imdb.title.akas.csv.gz',
 'zippedData\\imdb.title.basics.csv.gz',
 'zippedData\\imdb.title.crew.csv.gz',
 'zippedData\\imdb.title.principals.csv.gz',
 'zippedData\\imdb.title.ratings.csv.gz',
 'zippedData\\tmdb.movies.csv.gz',
 'zippedData\\tn.movie_budgets.csv.gz']

In [3]:
## Looping through individual data files

## Code adapted from James Irving
## Source: youtube.com/watch?v=rufvTgBEYN8&list=PLFknVelSJiSxSwXifV_ysDg50fzbuTzVt&index=41

clean_file_names = {}
split = '-----'*25

for file in data_files:
    name = file.replace('.csv.gz','').split('\\')[-1].replace('.','_')
    print(split)
    
    print(f"Preview of {name}:")
    clean_file_names[name] = pd.read_csv(file)
    display(clean_file_names[name].head(5))
    print()

-----------------------------------------------------------------------------------------------------------------------------
Preview of bom_movie_gross:


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.00,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.00,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.00,664300000,2010
3,Inception,WB,292600000.00,535700000,2010
4,Shrek Forever After,P/DW,238700000.00,513900000,2010



-----------------------------------------------------------------------------------------------------------------------------
Preview of imdb_name_basics:


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,nan,nan,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,nan,nan,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,nan,nan,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,nan,nan,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,nan,nan,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"



-----------------------------------------------------------------------------------------------------------------------------
Preview of imdb_title_akas:


...

,tconst,averagerating,numvotes
0,tt10356526,8.30,31
1,tt10384606,8.90,559
2,tt1042974,6.40,20
3,tt1043726,4.20,50352
4,tt1060240,6.50,21



-----------------------------------------------------------------------------------------------------------------------------
Preview of tmdb_movies:


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.53,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.70,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.73,2010-03-26,How to Train Your Dragon,7.70,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.52,2010-05-07,Iron Man 2,6.80,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.00,1995-11-22,Toy Story,7.90,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.92,2010-07-16,Inception,8.30,22186



-----------------------------------------------------------------------------------------------------------------------------
Preview of tn_movie_budgets:


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


# **Data Cleaning**

## Slicing DataFrames from Files

In [4]:
## Selecting for genre details
title_basics = clean_file_names['imdb_title_basics']
title_basics

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.00,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.00,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.00,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,nan,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.00,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.00,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,nan,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,nan,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.00,NaN


In [5]:
## Selecting for budget and gross details
movie_basics = clean_file_names['tn_movie_budgets']
movie_basics

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [6]:
## Selecting for gross details
movie_gross = clean_file_names['bom_movie_gross']
movie_gross

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.00,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.00,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.00,664300000,2010
3,Inception,WB,292600000.00,535700000,2010
4,Shrek Forever After,P/DW,238700000.00,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.00,NaN,2018
3383,Edward II (2018 re-release),FM,4800.00,NaN,2018
3384,El Pacto,Sony,2500.00,NaN,2018
3385,The Swan,Synergetic,2400.00,NaN,2018


In [48]:
## Selecting all titles for movies based on ID

title_akas = clean_file_names['imdb_title_akas']
title_akas

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.00
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.00
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.00
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.00
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.00
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,NaN,NaN,original,NaN,1.00
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,NaN,0.00
331700,tt9880178,1,La atención,NaN,NaN,original,NaN,1.00
331701,tt9880178,2,La atención,ES,NaN,NaN,NaN,0.00


In [113]:
## Selecting for tmdb_movies

tmdb_movies= clean_file_names['tmdb_movies']
tmdb_movies

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.53,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.70,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.73,2010-03-26,How to Train Your Dragon,7.70,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.52,2010-05-07,Iron Man 2,6.80,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.00,1995-11-22,Toy Story,7.90,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.92,2010-07-16,Inception,8.30,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.60,2018-10-13,Laboratory Conditions,0.00,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.60,2018-05-01,_EXHIBIT_84xxx_,0.00,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.60,2018-10-01,The Last One,0.00,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.60,2018-06-22,Trailer Made,0.00,1


Looks like this dataframe has a weird column that duplicated the index. I will drop that column.

In [117]:
display(tmdb_movies.columns)
display(tmdb_movies.columns[0])

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')

'Unnamed: 0'

In [127]:
## Dropping column

try:
    tmdb_movies.drop('Unnamed: 0', axis=1, inplace=True)
except:
    pass

## Testing results
'Unnamed: 0' not in tmdb_movies.columns

True

In [128]:
tmdb_movies

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.53,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.70,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.73,2010-03-26,How to Train Your Dragon,7.70,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.52,2010-05-07,Iron Man 2,6.80,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.00,1995-11-22,Toy Story,7.90,10174
4,"[28, 878, 12]",27205,en,Inception,27.92,2010-07-16,Inception,8.30,22186
...,...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,Laboratory Conditions,0.60,2018-10-13,Laboratory Conditions,0.00,1
26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.60,2018-05-01,_EXHIBIT_84xxx_,0.00,1
26514,"[14, 28, 12]",381231,en,The Last One,0.60,2018-10-01,The Last One,0.00,1
26515,"[10751, 12, 28]",366854,en,Trailer Made,0.60,2018-06-22,Trailer Made,0.00,1


## Merging Dataframes

In [7]:
## Merging title_basics and movie_basics on primary title

merged_basics_primary = pd.merge(title_basics,movie_basics, 
                                 left_on= 'primary_title', right_on= 'movie')
merged_basics_primary

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,tt0249516,Foodfight!,Foodfight!,2012,91.00,"Action,Animation,Comedy",26,"Dec 31, 2012",Foodfight!,"$45,000,000",$0,"$73,706"
1,tt0293429,Mortal Kombat,Mortal Kombat,2021,nan,"Action,Adventure,Fantasy",10,"Aug 18, 1995",Mortal Kombat,"$20,000,000","$70,433,227","$122,133,227"
2,tt0326592,The Overnight,The Overnight,2010,88.00,NaN,21,"Jun 19, 2015",The Overnight,"$200,000","$1,109,808","$1,165,996"
3,tt3844362,The Overnight,The Overnight,2015,79.00,"Comedy,Mystery",21,"Jun 19, 2015",The Overnight,"$200,000","$1,109,808","$1,165,996"
4,tt0337692,On the Road,On the Road,2012,124.00,"Adventure,Drama,Romance",17,"Mar 22, 2013",On the Road,"$25,000,000","$720,828","$9,313,302"
...,...,...,...,...,...,...,...,...,...,...,...,...
3810,tt9678962,Fuel,Fuel,2018,nan,"Documentary,Sport",35,"Nov 14, 2008",Fuel,"$2,500,000","$174,255","$174,255"
3811,tt9729206,Diner,Diner,2019,nan,Crime,11,"Apr 2, 1982",Diner,"$5,000,000","$12,592,907","$12,592,907"
3812,tt9805168,Traitor,Traitor,2015,110.00,"Action,Drama,Romance",25,"Aug 27, 2008",Traitor,"$22,000,000","$23,530,831","$27,882,226"
3813,tt9844102,Ray,Ray,2018,111.00,Crime,8,"Oct 29, 2004",Ray,"$40,000,000","$75,305,995","$124,823,094"


In [9]:
df = merged_basics_primary.copy()

### Removing Redundant "Movie" Column

In [11]:
## Verifying all titles/movies match - np.where faster than logical slicing

for x in np.where((df.loc[:,'primary_title'] != df.loc[:,'movie']),1,0):
    if x == 1:
        print(x)

In [12]:
## Dropping redundant "movie" column -  incl t/e to be able to rerun notebook

for col in df.columns:
    try:
        df.drop('movie', axis=1, inplace=True)
    except:
        pass
    
## Confirming removal
'movie' in df.columns

False

## Converting Currencies from Str to Int

In [13]:
## Converting gross amounts from strings to integers and removing 

dollar_to_int = ['production_budget','worldwide_gross','domestic_gross']

for i in dollar_to_int:
    try:
        df[i] = df[i].map((lambda x: int(x.replace('$','').replace(',',''))))

    except Exception:
        print('---'*25)
        print('Already converted.')
        print()
        
    display(df[i][:3])

0    45000000
1    20000000
2      200000
Name: production_budget, dtype: int64

0        73706
1    122133227
2      1165996
Name: worldwide_gross, dtype: int64

0           0
1    70433227
2     1109808
Name: domestic_gross, dtype: int64

## Handling Null Values

In [18]:
df_filled = df.copy()

### identifying Null Values

In [14]:
## Identify the total number of missing values for the dataframe

display(df.isna().sum())

tconst                 0
primary_title          0
original_title         1
start_year             0
runtime_minutes      487
genres                72
id                     0
release_date           0
production_budget      0
domestic_gross         0
worldwide_gross        0
dtype: int64

In [15]:
## Creating function to diagnose data for null values and generate report
def check_null(df):
    '''Checks a given dataframe for null values in each column.
    
    If no null values, it prints out a confirmation of no null values.
    
    If null values are found, it generates a report a report indicating:
        * Column name
        * Datatype
        * If object: prints most frequent value and count
        * If int/float: prints mean, median of values for the column with null
        values'''
    
    col_nan = [pair[0] for pair in zip(df.columns, df.isna().sum()) if pair[1] != 0]
   
    if len(col_nan) >= 1:
        print()
        print('The columns with missing values is/are:')
        print()
        print('------')
    
    else:
        print('There are no null values.')

    for i in col_nan:

        print()
        print(f"** '{i}' **")
        print()
        print(f'Dataype: {df[i].dtype}')
        print()

        if df[i].dtype  == 'object':
            print(f'Mode: {df[i].value_counts().index[1]} ({df[i].value_counts()[1]}x)')
            print()
            print('------'*5)

        elif [(df[i].dtype  == 'int64') or df[i].dtype  == 'float64']:
            print(f'Mean: {df[i].mean():.2f}')
            print(f'Median: {df[i].median():.2f}')
            print()
            print('------'*5)

        else:
            print('------'*5)
            
            
    return

In [17]:
## Running function and confirming functionality

check_null(df)


The columns with missing values is/are:

------

** 'original_title' **

Dataype: object

Mode: The Gift (24x)

------------------------------

** 'runtime_minutes' **

Dataype: float64

Mean: 97.20
Median: 98.00

------------------------------

** 'genres' **

Dataype: object

Mode: Documentary (290x)

------------------------------


#### *Deprecated*

In [ ]:
## Deprecated

# def check_null_old(df):
#     col_nan = []
#     for pair in zip(df.columns, df.isna().sum()):
#         if pair[1] == 0:
#             pass
#         else:
#             col_nan.append(pair[0])

#     if len(col_nan) >= 1:
#         print()
#         print('The columns with missing values is/are:')
#         print()
#         print('------')

#     for i in col_nan:

#         print()
#         print(f"** '{i}' **")
#         print()
#         print(f'Dataype: {df[i].dtype}')
#         print()

#         if df[i].dtype  == 'object':
#             print(f'Mode: {df[i].value_counts().index[1]} ({df[i].value_counts()[1]}x)')
#             print()
#             print('------'*5)

#         elif [(df[i].dtype  == 'int64') or df[i].dtype  == 'float64']:
#             print(f'Mean: {df[i].mean():.2f}')
#             print(f'Median: {df[i].median():.2f}')
#             print()
#             print('------'*5)

#         else:
#             print('------'*5)
            
            
#     return

### Filling Missing `runtime_minutes`

In [20]:
## Calculating mean & median values to determine fill value for missing values

mean_runtime = df["runtime_minutes"].mean()

median_runtime = df["runtime_minutes"].median()

abs_diff = abs(mean_runtime-median_runtime)

print(f'The average and median runtimes are: {mean_runtime:,.2f} and {median_runtime:,.2f}')
print()
print(f'The difference between the average and median values is: {abs_diff:,.2f}')

The average and median runtimes are: 97.20 and 98.00

The difference between the average and median values is: 0.80


>Based on the minimal difference in values, either value would be a valid representation of the overall data.
>
>I will select the average value to fill for missing values as we don't have significant outliers. Outliers would increase the difference between the values and affect our later inferences.

In [21]:
## Filling missing value with the pre-calcuated average

df_filled['runtime_minutes'].fillna(mean_runtime, inplace=True)

In [44]:
## Confirming the value was filled.

df_filled['original_title'].isna().sum()

0

In [45]:
## Checking remaining columns with null values

check_null(df_filled)


The columns with missing values is/are:

------

** 'genres' **

Dataype: object

Mode: Documentary (290x)

------------------------------


### Filling `original_title`

In [25]:
## Identifying missing value

df_filled[df_filled['original_title'].isna()]

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,id,release_date,production_budget,domestic_gross,worldwide_gross
447,tt4732998,Limbo,NaN,2017,97.20,NaN,55,"Jun 4, 1999",8300000,2160710,2598224


> Looks like this feature is only missing one value. I will try to use the "titles_aka" dataframe from the original datasets to check if there was a different name. Otherwise, I will use the primary title.

#### Trying the 'title_akas" dataframe

In [86]:
## Identifying ID of missing movie

id_missing_title = df_filled.iloc[447][0]
id_missing_title

'tt4732998'

In [104]:
## Checking to see if the movie is in the table.

id_missing_title in title_akas['title_id'].values

False

> Looks like the ID is not in the "title_akas" dataframe. I will use the primary title instead.

#### Using Primary title

In [37]:
## Slicing out the titles for replacement

df_filled.iloc[447][1:3]

primary_title     Limbo
original_title      NaN
Name: 447, dtype: object

In [39]:
## Creating variable for fillna()

fill_title = df_filled.iloc[447][1]
fill_title

'Limbo'

In [41]:
## Filling missing value with title

df_filled['original_title'].fillna('fill_title', inplace=True)

In [42]:
## Confirming the value was filled

df_filled['original_title'].isna().sum()

0

#### Summary

> I identified the single missing null value for the "original_title" column.
>
> I decided to fill the value with the "primary_title" value as many other columns in the dataframe shared primary/original titles as well.
>
> I used .iloc indexing to slice out the value and then filled the missing value via the .fillna() method
>
> The final result is a filled column.

In [43]:
## Checking remaining columns with null values

check_null(df_filled)


The columns with missing values is/are:

------

** 'genres' **

Dataype: object

Mode: Documentary (290x)

------------------------------


### Filling `genres`

In [47]:
## Identifying missing value

df_filled[df_filled['genres'].isna()]

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,id,release_date,production_budget,domestic_gross,worldwide_gross
2,tt0326592,The Overnight,The Overnight,2010,88.00,NaN,21,"Jun 19, 2015",200000,1109808,1165996
109,tt6634536,Red Dog,Red Dog,2017,97.20,NaN,67,"Dec 31, 2011",9100000,0,20927794
180,tt6858500,Robin Hood,Robin Hood,2018,97.20,NaN,39,"May 14, 2010",210000000,105487148,322459006
181,tt6858500,Robin Hood,Robin Hood,2018,97.20,NaN,9,"Nov 21, 2018",99000000,30824628,84747441
217,tt4532628,The Promise,Ghoul,2014,75.00,NaN,78,"Apr 21, 2017",90000000,8224288,10551417
...,...,...,...,...,...,...,...,...,...,...,...
3675,tt6448350,Shine a Light,Shine a Light,2010,97.20,NaN,34,"Apr 4, 2008",1000000,5505267,16173879
3690,tt6612954,The Dress,The Dress,2017,127.00,NaN,99,"Jan 16, 1998",2650000,16556,16556
3691,tt6616538,Bang,Bang,2015,97.20,NaN,70,"Apr 1, 1996",10000,527,527
3709,tt6962340,The Touch,The Touch,2017,48.00,NaN,10,"Mar 31, 2004",20000000,0,5918742


> This series of missing values is much more difficult to fill versus the other three as I don't have any other rows to reference, and the datatype for the value would be a string, so I can't use mean/median.

# **Feature Engineering**

## Creating Seasons and Quarters

### Datetime Approach

In [ ]:
## Changing release date to datetime datatype

df_filled['release_datetime'] = pd.to_datetime(df_filled['release_date'])
df_filled['release_datetime']

In [ ]:
## Using datetime dtype to create months column
df_filled['release_month_dt'] = df_filled['release_datetime'].dt.month_name()
df_filled['release_month_dt']

In [ ]:
df_filled['release_quarter'] = df_filled['release_datetime'].dt.quarter
df_filled['release_quarter']

### Manual Approach

In [ ]:
## Inspecting original values
df_filled['release_date'][:5]

In [ ]:
## Pulling month
test_month = df_filled['release_date'][0][:3]
test_month

In [ ]:
## Creating new column for the month of each release date
release_month = []

for movie in df_filled['release_date']:
    release_month.append(movie[:3])
    
df_filled['release_month_manual'] = release_month

In [ ]:
## Using map and lambda functions to slice out month from string
df_filled['release_month_manual'] = df_filled['release_date'].map(lambda x: x[:3])
df_filled['release_month_manual']

In [ ]:
## Creating seasons based on meteorological definitions of each season
season = []

for month in df_filled['release_month_manual']:
    if month == 'Jan':
        season.append('Winter')
    elif month == 'Feb':
        season.append('Winter')
    elif month == 'Mar':
        season.append('Spring')
    elif month == 'Apr':
        season.append('Spring')
    elif month == 'May':
        season.append('Spring')
    elif month == 'Jun':
        season.append('Summer')
    elif month == 'Jul':
        season.append('Summer')
    elif month == 'Aug':
        season.append('Summer')
    elif month == 'Sep':
        season.append('Fall')
    elif month == 'Oct':
        season.append('Fall')
    elif month == 'Nov':
        season.append('Fall')
    elif month == 'Dec':
        season.append('Winter')
    else:
        print('na')

df_filled['release_season_manual'] = season
df_filled['release_season_manual']

### Reviewing Changes

In [ ]:
df_filled

## Creating Profit and ROI

In [ ]:
df_filled['profit'] = df_filled['worldwide_gross'] - df_filled['production_budget']
df_filled['profit'] 

In [ ]:
df_filled['ROI'] = (df_filled['worldwide_gross'] - df_filled['production_budget'])/df_filled['production_budget']
df_filled['ROI']

# **Analysis**

## Q1: What genres perform the best?

### Splitting Genres into Lists

In [ ]:
# ## Via map & lambda - slower than .str
# df_filled['genres_list'] = df_filled['genres'].map(lambda x: x.split(','))
# df_filled

In [ ]:
## Via .str and string methods (faster than map/lambda)
df_filled['genres_str'] = df_filled['genres'].str.title().str.split(',')
df_filled['genres_str']

# **Visualizations**

In [ ]:
## Creating separate df for plotting with each genre on a separate row
plot_df = df_filled.explode('genres_str')
plot_df

## Total Worldwide Gross per Genre

In [ ]:
## Defining a function to perform aggregation and plot the data

def sort_genres_sum(df, col_to_sort, agg='sum', verbose=True):
    '''Creates a new dataframe from a given dataframe created by the sum of
    all values for each genre, then sorted by the specified column name.
    
    Args & kwargs:
    df - original dataframe to be sorted
    col_to_sort - column name (str) by which to sort the data
    '''
    ## Adding functionality to handle different aggregation methods
    if agg == 'sum':
        agg_func = np.sum
        agg_title = 'Total'
    elif agg == 'median':
        agg_func = np.median
        agg_title = 'Median'
    else:
        agg_func = np.mean
        agg_title = 'Average'
    
    ## Determining performance rankings and creating a list of names for viz
    idx = list(df.groupby("genres_str", as_index=True).agg(agg_func).sort_values(by = col_to_sort, ascending=False).index)
    
    ## Graphing results of agg function ordered by index (above)
    fig = plt.figure(figsize=(15,4))
    sns.barplot(data=df, x= 'genres_str', y= col_to_sort, order = idx, estimator = agg_func)
    plt.xticks(rotation=45, ha= 'right')
    plt.suptitle(f'Total {col_to_sort.title().replace("_", " ")} by Genre')
    plt.xlabel('Genres')
    plt.ylabel(f'Total {col_to_sort.title().replace("_", " ")}');
    
    ## Adding option to turn off dataframe showing results
    if verbose == True:
        display(df.sort_values(col_to_sort, ascending=False))
    
    return fig

In [ ]:
# df_filled.columns

In [ ]:
# df_filled.sort_values('domestic_gross')

In [ ]:
## Testing Function

sort_genres_sum(plot_df, col_to_sort='worldwide_gross', agg='sum');

**Observations:**
>* **Top three genres:** Adventure, Action, Drama
>* **Lowest three genres:** Western, News, Reality-TV
>* On this graph, musicals are ranked much lower due to their smaller total gross box office sales (versus calculating the average of all gross values).

**Suggestions**
>* **Safest Genres (by Gross)** are the top three genres
    * Lowest points on the error bars indicate high performance even at their worst.
>* **Select from:** Action, Animation, Adventure, Fantasy, Sci-Fi, Family, or Musicals
    * All others show poor performance

## Average Gross per Genre

In [ ]:
## Defining a function to perform aggregation and plot the data

def sort_genres_avg(df, col_to_sort):
    '''Creates a new dataframe from a given dataframe created by the average of
    all values for each genre, then sorted by the specified column name.
    
    Args & kwargs:
    df - original dataframe to be sorted
    col_to_sort - column name (str) by which to sort the data"'''
    
    df1 = df.groupby("genres_str").mean().reset_index().sort_values(by = col_to_sort, ascending=False)

    plt.figure(figsize=(15,4))
    sns.barplot(data=df1, x= 'genres_str', y= col_to_sort )
    plt.xticks(rotation=45, ha= 'right')
    plt.suptitle('Average Worldwide Gross by Genre')
    plt.xlabel('Genres')
    plt.ylabel('Total Worldwide Gross ($)');
    
    return df1.head(5)

In [ ]:
sort_genres_avg(plot_df, col_to_sort='worldwide_gross')

#### Old Code - Saving just in case

In [ ]:
# ## Visualizing sorted sum 

# df_sorted = sort_genres_sum(plot_df, 'worldwide_gross')

# plt.figure(figsize=(15,4))
# sns.barplot(data=df_sorted, x= 'genres_str', y='worldwide_gross')
# plt.xticks(rotation=45, ha= 'right')
# plt.suptitle('Total Worldwide Gross by Genre')
# plt.xlabel('Genres')
# plt.ylabel('Total Worldwide Gross ($)');

In [ ]:
# ## Visualizing sorted average 

# avg_sorted = sort_genres_avg(plot_df, 'worldwide_gross')

# plt.figure(figsize=(15,4))
# sns.barplot(data=avg_sorted, x= 'genres_str', y='worldwide_gross')
# plt.xticks(rotation=45, ha= 'right')
# plt.suptitle('Total Worldwide Gross by Genre')
# plt.xlabel('Genres')
# plt.ylabel('Total Worldwide Gross ($)');

In [ ]:
# def sort_genres_avg(df, col_to_sort):
#     '''Creates a new dataframe from a given dataframe created by the average of
#     all values for each genre, then sorted by the specified column name.
    
#     Args & kwargs:
#     df - original dataframe to be sorted
#     col_to_sort - column name (str) by which to sort the data"'''
    
#     df1 = df.groupby("genres_str").agg('mean').reset_index().sort_values(by = col_to_sort, ascending=False)
    
#     return df1

In [ ]:
# ## First version of visualizing results - 

# plt.figure(figsize=(15,4))
# sns.barplot(data=plot_df, x= 'genres_str', y='worldwide_gross')#, order=sum_gross_ww)
# plt.xticks(rotation=45, ha= 'right')
# plt.suptitle('Worldwide Gross by Genre')
# plt.xlabel('Genres')
# plt.ylabel('Worldwide Gross ($)');

In [ ]:
# ## Not working - attempted to create function to sort and visualize
# def visualize_metrics(df, col_to_sort, sort_order):
    
#     df_sorted = df.groupby("genres_str").sum().reset_index().sort_values(by = col_to_sort, ascending=False)
    
#     plt.figure(figsize=(15,4))
#     sns.barplot(data=df_sorted, x= 'genres_str', y=col_to_sort, order=sort_order)
#     plt.xticks(rotation=45, ha= 'right')
#     plt.suptitle('Worldwide Gross by Genre')
#     plt.xlabel('Genres')
#     plt.ylabel(col_to_sort);
    
#     return

## ROI per Genre

In [ ]:
## Creating list of genres based on average ROI for graphing purposes

mean_roi = list(plot_df.groupby("genres_str").mean()['ROI'].sort_values(ascending=False).index)
# mean_roi

In [ ]:
## Visualizing Average Worldwide Gross per Genre

plt.figure(figsize=(15,4))
sns.barplot(data=plot_df, x= 'genres_str', y='ROI', order=mean_roi)
plt.xticks(rotation=45, ha= 'right')
plt.suptitle('Average Return-on-Investment by Genre')
plt.xlabel('Genres')
plt.ylabel('ROI (%)');

**Observations:**
>* **Top three genres:** Animation, Adventure, and Sci-Fi
>* **Lowest three genres:** Reality-TV, War and News
>* *Musicals are a high-risk, high-reward option*
    * Their gross can exceed Animation, or fall below the top 5 genres.

**Suggestions**
>* **Safest Genres (by Gross)** are the top three genres
    * Lowest points on the error bars indicate high gross performance even at their worst
>* Select from Action, Animation, Adventure, Fantasy, Sci-Fi, Family, or Musicals
    * All others show poor performance

### **A1: Top Genres**

>* List mean, roi for evaluations

## Visualizing Seasonal Performance

### Seasonal Performance - All Movies (ROI)

In [ ]:
## Creating basic overview
g = sns.barplot(data=plot_df, x='release_season_manual', y='ROI', 
                order=['Spring', 'Summer', 'Fall', 'Winter'], 
                estimator=np.mean)
g.set_xlabel('Seasons')
g.set_ylabel('ROI(%)')
g.set_title('Seasonal Performance (All Movies)');

### Seasonal Performance - All Movies (Ww Gross)

In [ ]:
## Creating basic overview
g = sns.barplot(data=plot_df, x='release_season_manual', y='worldwide_gross', 
                order=['Spring', 'Summer', 'Fall', 'Winter'], 
                estimator=np.mean)
g.set_xlabel('Seasons')
g.set_ylabel('Worldwide Gross')
g.set_title('Seasonal Performance (All Movies)');

**Observations:**
>* Summer is the best season for releases, with winter being the worst time.
>* Summer and Spring seasons seem to be the most productive seasons.
>* Fall and Winter perform worse.

**Suggestions**
>* Focus release times in Summer/Spring
>* Avoid Fall/Winter

### Genre Performance by Season

In [ ]:
## Visualizing each genre's performance by season
g = sns.catplot(data=plot_df, col='release_season_manual',
            y='ROI', kind='bar', x='genres_str', col_wrap=2, 
            aspect=1.75, order=mean_roi)
(g.set_axis_labels('Category', 'ROI (%)')
 .set_xticklabels(rotation=45)
 .set_titles("{col_name}"))
 
plt.tight_layout();

In [ ]:
## Visualizing each genre's performance by season
g = sns.catplot(data=plot_df, col='release_season_manual',
            y='worldwide_gross', kind='bar', x='genres_str', col_wrap=2, 
            aspect=1.75)#, sharex=False)
(g.set_axis_labels('Category', 'Worldwide Gross ($)')
 .set_xticklabels(rotation=45)
 .set_titles("{col_name}"))
 
plt.tight_layout();

**Observations:**
>* Springtime releases show highest gross performance on average
>* Wintertime shows lowest performances across all genres
>* The results match up with our overall view for all genres

**Suggestions**
>* To maximize profitability of musicals, release in spring
>* Avoid releasing Animations in the winter - all other seasons perform better
>* Avoid releasing news-related movies in the Spring

## Seasonal Performance -  Insights

**Observations:**
>
>The top five genres tend to perform relatively well regardless of the season with little difference between each season.
>
> Musicals show a strongest performance in the springtime - it is only worthwhile to release a musical in the spring.
>
**Questions**
> 
>What is the profitability and return on investment for each genre?

## Quarterly Performance

How would the data look when comparing seasons to quarters?

In [ ]:
## Sorting by release quarters for graphing
df_filled.sort_values('release_quarter', inplace=True)
df_filled.reset_index(drop=True, inplace=True)
# df_filled

In [ ]:
## Creating basic overview
g = sns.barplot(data=plot_df, x='release_quarter', y='worldwide_gross')
g.set_xlabel('Quarters')
g.set_ylabel('Worldwide Gross')
g.set_title('Quarterly Performance (All Movies)');

**Observations:**
>* 
>* 
>* 

**Suggestions**
>* 
>* 

In [ ]:
## Generating figure for quarterly performance breakdown
g = sns.catplot(data=plot_df, col='release_quarter',
            y='worldwide_gross', kind='bar', x='genres_str', col_wrap=2, 
            aspect=1.75)#, sharex=False)
(g.set_axis_labels('Category', 'Worldwide Gross ($)')
 .set_xticklabels(rotation=45)
 .set_titles("Quarter {col_name}"))
 
plt.tight_layout();

**Observations:**
>* 
>* 
>* 

**Suggestions**
>* 
>* 

In [ ]:
# g = sns.catplot(x = 'genres_list', y='worldwide_gross', 
#                hue = 'release_quarter',data=plot_df, kind='bar', aspect=3.65)
# g.set_xticklabels(rotation=45);

In [ ]:
# g = sns.catplot(x = 'genres_list', y='worldwide_gross',
#                 hue = 'release_season_manual',data=plot_df, kind='bar',
#                 aspect=3.65)
# g.set_xticklabels(rotation=45);

## **Comparing S & Q**

In [ ]:
## Comparing Seasonal/Quarterly breakdowns

fig, axes = plt.subplots(nrows=2, figsize=(7,7))
sns.barplot(x = 'release_season_manual', y='worldwide_gross',data=plot_df, 
            ax=axes[0], order=['Spring', 'Summer', 'Fall', 'Winter'])
sns.barplot(x = 'release_quarter', y='worldwide_gross', data=plot_df,
            ax=axes[1])

## Changing settings
axes[0].set_xlabel("Seasons")
axes[0].set_ylabel('Worldwide Gross')

axes[1].set_xlabel("Quarters")
axes[1].set_ylabel('Worldwide Gross')

plt.tight_layout();

In [ ]:
## Creating single visualization to compare seasonal and quarterly performances

fig, axes = plt.subplots(nrows=2, figsize=(17,10))
sns.barplot(x = 'genres_str', y='worldwide_gross',
            hue = 'release_season_manual',data=plot_df, ax=axes[0])
sns.barplot(x = 'genres_str', y='worldwide_gross', hue = 'release_quarter',
            data=plot_df, ax=axes[1])

## Changing settings
axes[0].set_xticklabels(axes[0].get_xticklabels(),rotation=45, ha='right')
axes[0].set_xlabel("Genres")
axes[0].set_ylabel('Worldwide Gross')
axes[0].legend(title='Seasons')

axes[1].set_xticklabels(axes[1].get_xticklabels(),rotation=45, ha='right')
axes[1].set_xlabel("Genres")
axes[1].set_ylabel('Worldwide Gross')
axes[1].legend(title='Quarters')

plt.tight_layout();

**Observations:**
>* 
>* 
>* 

**Suggestions**
>* 
>* 

# ✨ TODO

* Change to "quarterly"
* Move towards linreg (feature selection)

# Modeling Pre-processing

## OHE: Genres

In [ ]:
## Getting unique values for genres

unique_genres = plot_df['genres_str'].unique()
unique_genres

In [ ]:
# df_filled['family'] = (df_filled['genres_str'] == 'Family').astype(int)
# df_filled#['family']

In [ ]:
df_filled['genres'].str.contains('Family')

In [ ]:
## Manually creating OHE columns for each genre

for genre in unique_genres:
    df_filled[genre] = (df_filled['genres'].str.contains(genre)).astype(int)

In [ ]:
df_filled.columns

In [ ]:
try:
    df_filled.drop(columns=['genres', 'genres_str'], inplace=True)
except:
    pass

In [ ]:
df_filled

# Correlations/Multicollinearity

In [ ]:
# ## Code copied from P2proj

## Determining each feature's relationship with price

## 1. Drop categorical columns
## 2. Create correlation with target variable 
## 3. Display top/bottom 5 values

# df_corr = df.drop(['price', 'id', 'lat','long'], axis=1).corrwith(df['price']).sort_values(ascending=False)
# display(df_corr[0:5],df_corr[-6:-1])

In [ ]:
# ## Attempted to create a loop to check the datetype and add to a list for columns


# # for each runtime in the column, check if it is not a string
# #  if it is not, then zip that value with the value in the "primary_title" column
# #  then, add the zipped value to a list
# # finally, print the final list

# cat_col_list = []

# for i in col_list:
#     for n in range(len(col_list)):
#         if type(i[n]) == float:
# #             cat_col_list.append(i)
#             print(i)
        
# # cat_col_list

In [ ]:
## Realized I can't treat the 'runtime' column as either cat/col due to mixed dt w/in col

# display(list(df_filled.columns))

# display(list(df_filled.dtypes))

In [ ]:
## Identifying datatypes for each title to split into cat/cont/drop lists
cat_col = list(zip(list(df_filled.columns),list(df_filled.dtypes)))
cat_col

In [ ]:
## Breaking out each feature into either categorical/continuous/"to be dropped"

targets = ['profit','ROI']

cont_feat = ['start_year','production_budget']

cat_feat = ['release_month_dt','release_quarter','release_season_manual']

drop_feat = ['id','tconst','primary_title','original_title','release_datetime',
        'runtime_minutes','release_date','release_month_manual','domestic_gross','worldwide_gross']

In [ ]:
## Creating a new dataframe for correlation matrix

df_for_corr = df_filled.drop(columns = drop_feat).copy().reset_index(drop=True)
df_for_corr

In [ ]:
df_corr = df_for_corr.corrwith(df_for_corr['profit']).sort_values(ascending=False)
df_corr
# display(df_corr[0:5],df_corr[-6:-1])

In [ ]:
## Creating function to provide correlation matrix to determine multicollinearity

def calc_df_corr(df, list_cats, target_col):
    '''Identifying correlations within the data'''
    
    df_corr = df.drop(list_cats, axis=1).corrwith(df[target_col]).sort_values(ascending=False)
    
#     display(df_corr[0:5],df_corr[-6:-1])
    
    return df_corr

In [ ]:
calc_df_corr(df=df_for_corr, list_cats=cat_feat, target_col = 'profit')

# OHE

In [ ]:
## Save the features that we used previously - I will use these features 
## to fit my model.

X = df_filled[[*unique_genres,*cont_feat,*cat_feat]].copy()
X

In [ ]:
## Creating the y values by setting them equal to the 'price' values from the dataframe

y = df_filled['profit'].copy()
y

In [ ]:
## Verifying the two groups are of equal length

print(X.shape[0] == y.shape[0])

In [ ]:
## Establishing the train and test data before doing anything else

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=5)

In [ ]:
## Using OneHotEncoder to create columns for the categorical variables.

## Create the OHE without "drop='first" as it would throw an error in this case
ohe = OneHotEncoder(drop = 'first', sparse=False)

## Using OHE on our categorical variables for training (NOT testing)
train_ohe = ohe.fit_transform(X_train[cat_feat])
test_ohe = ohe.transform(X_test[cat_feat])

In [ ]:
## Getting feature names from our list of categories
feat_col_name = ohe.get_feature_names(cat_feat)

In [ ]:
## Creating dataframes from the results of our fit&transform and transform

train_ohe_df = pd.DataFrame(train_ohe, columns=feat_col_name, index=X_train.index)

test_ohe_df = pd.DataFrame(test_ohe, columns=feat_col_name, index=X_test.index)

In [ ]:
## Reviewing one of the resulting dataframes
test_ohe_df

In [ ]:
## Concatenating the two training dataframes after OHE
X_train_ohe = pd.concat([X_train.drop(cat_feat, axis=1), train_ohe_df], axis=1)

In [ ]:
X_train_ohe

In [ ]:
## Concatenating the two test dataframes after OHE
X_test_ohe = pd.concat([X_test.drop(cat_feat, axis=1), test_ohe_df], axis=1)
X_test_ohe

In [ ]:
## Instantiating the model and fitting it
lr = LinearRegression()

lr.fit(X_train_ohe, y_train)

In [ ]:
## Checking the R^2 for train and test

train_r = lr.score(X_train_ohe, y_train)
print(f'R-Square value for training data is {round(train_r,3)}.')

test_r = lr.score(X_test_ohe, y_test)
print(f'R-Square value for test data is {round(test_r,3)}.')

## Getting model coefficients
train_coef = pd.Series(lr.coef_, index=X_train_ohe.columns)
train_coef['intercept'] = lr.intercept_

## Displaying resulting features and coefficients
train_coef.sort_values(ascending=False)#.plot(kind='barh');

# Creating Statsmodels Model

In [ ]:
def diagnose_model(model, figsize=(10,5)):
    """ ---
    
    Argument:
        * model: provide the linear regression model for diagnostics
    
    Keyword Argument:
        * figsize: default (10,5); can increase/decrease for larger/smaller
    ---
    
    * Display the summary details of the provided model
    * Create two scatter plots to test assumptions of linearity
        * Predictions: verifying homoscedasticity (no cone-shapes)
        * Residuals: confirming normal distribution of residuals
    ---
    
    """
    display(model.summary())
    
    fig, axes = plt.subplots(ncols=2, figsize=figsize)

    axes[0].scatter(model.predict(), model.resid)
    axes[0].axhline()
    axes[0].set_xlabel('Model Predictions')
    axes[0].set_ylabel('Model Residuals')
    axes[0].set_title('Testing for Homoscedasticity')

    sms.graphics.qqplot(data=model.resid, fit=True, line = "45", ax=axes[1])
    
    plt.tight_layout()
    
    return

In [ ]:
X_train_ohe = sms.add_constant(X_train_ohe)

In [ ]:
X_test_ohe = sms.add_constant(X_test_ohe)

In [ ]:
model = sms.OLS(y_train, X_train_ohe).fit()

diagnose_model(model)

In [ ]:
sns.displot(y_train, aspect=2);

* Outliers
* Adding features from other df's at start

# Statistical Testing

# Inferential Modeling

# Predictive Modeling

# Final Results